In [ ]:
import overpy
import fiona
from fiona.crs import from_epsg
from pyproj import Geod
from shapely.geometry import Polygon, mapping


### Download building footprints from OSM

In [2]:
api = overpy.Overpass()

def query(bbox, limit=100):
    result = api.query(
        f"""
            way["building"]({','.join(map(str, bbox))});
            out geom {limit};
        """)
    return result

In [24]:
BBOX_BERLIN = (52.368053,13.103943,52.653895,13.686218) # test with bounding box of Berlin

all_buildings = query(bbox=BBOX_BERLIN, limit=500)
print(f'{len(all_buildings.ways)} buildings fetched from OSM')

500 buildings fetched from OSM


### Prepare dataset

In [35]:
from shapely import box

filtered_buildings = []
geod = Geod(ellps="WGS84")

for way in all_buildings.ways:
    poly = Polygon([(float(node.lon), float(node.lat)) for node in way.get_nodes(resolve_missing=True)])
    bounds = box(*poly.bounds)
    bounds_area, bounds_perimeter = geod.geometry_area_perimeter(bounds)

    bounds_area = abs(bounds_area) # take absolute value as polygon area can be negative if polygon is not-ccw

    # this gets rid of buildings with square area below 250m2 or above 500m2 to get more similar data
    # we should replace this with a better measure perhaps
    if bounds_area > 500 or bounds_area < 250: 
        continue

    filtered_buildings.append(poly)

print(f'{len(filtered_buildings)} buildings after filtering')

116 buildings after filtering


### Export data to shapefile

This exports the previously collected geometries to a shapefile using fiona. The shapefile will be created in the current directory.

In [32]:
schema = {'geometry': 'Polygon', 'properties': {}}
shape_out = "buildings.shp"
with fiona.open(shape_out, 'w', crs='epsg:4326', driver='ESRI Shapefile', schema=schema) as output:
    for poly in filtered_buildings:
        output.write({'geometry': mapping(poly)})  